In [15]:
import cv2
import numpy as np

# 이미지 불러오기
image = cv2.imread('E:/glass_git/ML-DL/vision/test2.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
binary = cv2.adaptiveThreshold(gray, 255,
                               cv2.ADAPTIVE_THRESH_MEAN_C,
                               cv2.THRESH_BINARY_INV, 11, 3)
# 경계선 검출을 위한 이진화
blurred = cv2.GaussianBlur(binary, (5, 5), 0)
edged = cv2.Canny(blurred, 30, 100)


# 윤곽선 찾기
contours, _ = cv2.findContours(binary.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 가장 큰 사각형 윤곽선 찾기
max_area = 0
best_rect = None
for cnt in contours:
    approx = cv2.approxPolyDP(cnt, 0.02 * cv2.arcLength(cnt, True), True)
    if len(approx) == 4:  # 사각형 여부 확인
        area = cv2.contourArea(cnt)
        if area > max_area:
            max_area = area
            best_rect = approx

# 중심 좌표 계산 및 시각화
if best_rect is not None:
    M = cv2.moments(best_rect)
    if M["m00"] != 0:
        cx = int(M["m10"] / M["m00"])
        cy = int(M["m01"] / M["m00"])
        
        # 중심점 시각화
        annotated = image.copy()
        cv2.circle(annotated, (cx, cy), 5, (0, 0, 255), -1)  # 빨간 점
        cv2.putText(annotated, f"({cx}, {cy})", (cx + 10, cy - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
        cv2.drawContours(annotated, [best_rect], -1, (0, 255, 0), 2)  # 윤곽선도 표시

        # 이미지 저장 또는 표시
        cv2.imwrite('E:/glass_git/ML-DL/vision/test_detect2.jpg', annotated)
        print(f"중심 좌표: ({cx}, {cy}) - 시각화된 이미지 저장됨: center_annotated.png")
    else:
        print("중심 좌표를 계산할 수 없습니다.")
else:
    print("사각형을 찾을 수 없습니다.")


중심 좌표: (357, 425) - 시각화된 이미지 저장됨: center_annotated.png


In [17]:
import cv2
import numpy as np

# 이미지 불러오기
image = cv2.imread('E:/glass_git/ML-DL/vision/test2.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Binary 처리 + Morphology
binary = cv2.adaptiveThreshold(gray, 255,
                               cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                               cv2.THRESH_BINARY_INV, 11, 3)
kernel = np.ones((5, 5), np.uint8)
morph = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
morph = cv2.morphologyEx(morph, cv2.MORPH_OPEN, kernel)

# 윤곽선 검출
contours, _ = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 이미지 아래쪽에 가장 가까운 사각형 찾기
max_bottom_y = -1
best_rect = None

for cnt in contours:
    approx = cv2.approxPolyDP(cnt, 0.02 * cv2.arcLength(cnt, True), True)
    if len(approx) == 4 and cv2.isContourConvex(approx):
        area = cv2.contourArea(cnt)
        if area > 500:  # 너무 작은 건 제외
            bottom_y = max([pt[0][1] for pt in approx])  # y값 최대
            if bottom_y > max_bottom_y:
                max_bottom_y = bottom_y
                best_rect = approx

# 중심 좌표 계산 및 시각화
if best_rect is not None:
    M = cv2.moments(best_rect)
    if M["m00"] != 0:
        cx = int(M["m10"] / M["m00"])
        cy = int(M["m01"] / M["m00"])

        annotated = image.copy()
        cv2.drawContours(annotated, [best_rect], -1, (0, 255, 0), 2)
        cv2.circle(annotated, (cx, cy), 5, (0, 0, 255), -1)
        cv2.putText(annotated, f"({cx}, {cy})", (cx + 10, cy - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

        cv2.imwrite('E:/glass_git/ML-DL/vision/test_detect2.jpg', annotated)
        print(f"✅ 가장 아래쪽 사각형 중심 좌표: ({cx}, {cy}) - 저장됨")
    else:
        print("중심 좌표 계산 실패")
else:
    print("❌ 아래쪽 사각형 미검출")


❌ 아래쪽 사각형 미검출


In [12]:
import cv2
import numpy as np

# 이미지 불러오기
image = cv2.imread('E:/glass_git/ML-DL/vision/test.png')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# 대비 향상을 위한 adaptive threshold
thresh = cv2.adaptiveThreshold(gray, 255,
                               cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                               cv2.THRESH_BINARY_INV, 5, 2)

# 모폴로지 연산으로 노이즈 제거
kernel = np.ones((5,5), np.uint8)
morph = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

# 윤곽선 검출
contours, _ = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 가장 큰 사각형 윤곽선 찾기
max_area = 0
best_rect = None
for cnt in contours:
    approx = cv2.approxPolyDP(cnt, 0.02 * cv2.arcLength(cnt, True), True)
    if len(approx) == 4:
        area = cv2.contourArea(cnt)
        if area > max_area:
            max_area = area
            best_rect = approx

# 중심 좌표 계산 및 시각화
if best_rect is not None:
    M = cv2.moments(best_rect)
    if M["m00"] != 0:
        cx = int(M["m10"] / M["m00"])
        cy = int(M["m01"] / M["m00"])
        
        # 시각화
        annotated = image.copy()
        cv2.circle(annotated, (cx, cy), 5, (0, 0, 255), -1)
        cv2.putText(annotated, f"({cx}, {cy})", (cx + 10, cy - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
        cv2.drawContours(annotated, [best_rect], -1, (0, 255, 0), 2)

        # 결과 저장
        cv2.imwrite('E:/glass_git/ML-DL/vision/test_detect.png', annotated)
        print(f"✅ 중심 좌표: ({cx}, {cy}) - 개선된 시각화 저장됨: improved_center_annotated.png")
    else:
        print("중심 좌표 계산 불가.")
else:
    print("사각형 윤곽선 미검출.")


✅ 중심 좌표: (369, 69) - 개선된 시각화 저장됨: improved_center_annotated.png


In [8]:
import cv2
import numpy as np

# 이미지 로드
image = cv2.imread('E:/glass_git/ML-DL/vision/test.png')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# 대비 강조 + 이진화
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
                               cv2.THRESH_BINARY_INV, 11, 3)

# 잡음 제거
kernel = np.ones((5,5), np.uint8)
morph = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

# 윤곽선 검출
contours, _ = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 조건에 맞는 4각형 찾기
max_area = 0
best_quad = None

for cnt in contours:
    approx = cv2.approxPolyDP(cnt, 0.02 * cv2.arcLength(cnt, True), True)
    
    if len(approx) == 4 and cv2.isContourConvex(approx):
        area = cv2.contourArea(approx)
        if area > 500:  # 너무 작은 건 무시
            # 기울기 판단을 위해 꼭짓점 정렬
            pts = approx.reshape(4, 2)
            pts = sorted(pts, key=lambda x: x[1])  # y 기준 정렬
            top_pts = sorted(pts[:2], key=lambda x: x[0])
            bottom_pts = sorted(pts[2:], key=lambda x: x[0])
            ordered = np.array([top_pts[0], top_pts[1], bottom_pts[1], bottom_pts[0]])

            if area > max_area:
                max_area = area
                best_quad = ordered

# 중심 좌표 + 시각화
if best_quad is not None:
    M = cv2.moments(best_quad)
    cx = int(M["m10"] / M["m00"])
    cy = int(M["m01"] / M["m00"])
    
    annotated = image.copy()
    cv2.drawContours(annotated, [best_quad], -1, (0, 255, 0), 2)
    cv2.circle(annotated, (cx, cy), 5, (0, 0, 255), -1)
    cv2.putText(annotated, f"({cx}, {cy})", (cx + 10, cy - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

    # 저장
    cv2.imwrite('E:/glass_git/ML-DL/vision/test_detect.png', annotated)
    print(f"✅ 정면(사다리꼴 포함) 중심 좌표: ({cx}, {cy}) 저장됨: final_detected_front_face.png")
else:
    print("❌ 정면 윤곽을 찾을 수 없습니다.")


❌ 정면 윤곽을 찾을 수 없습니다.


In [16]:
import cv2
import numpy as np

# 이미지 로드
image = cv2.imread('E:/glass_git/ML-DL/vision/test.png')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Binary 처리 (adaptive threshold → 명확한 흑/백)
binary = cv2.adaptiveThreshold(gray, 255,
                               cv2.ADAPTIVE_THRESH_MEAN_C,
                               cv2.THRESH_BINARY_INV, 11, 3)

# Morphology 연산으로 노이즈 제거
kernel = np.ones((5,5), np.uint8)
morph = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)

# 윤곽선 추출
contours, _ = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 가장 큰 4각형 찾기 (사다리꼴 포함)
max_area = 0
best_quad = None

for cnt in contours:
    approx = cv2.approxPolyDP(cnt, 0.02 * cv2.arcLength(cnt, True), True)
    if len(approx) == 4 and cv2.isContourConvex(approx):
        area = cv2.contourArea(approx)
        if area > 1000:
            if area > max_area:
                max_area = area
                best_quad = approx.reshape(4, 2)

# 중심 좌표 계산 및 시각화
if best_quad is not None:
    M = cv2.moments(best_quad)
    cx = int(M["m10"] / M["m00"])
    cy = int(M["m01"] / M["m00"])

    annotated = image.copy()
    cv2.drawContours(annotated, [best_quad], -1, (0, 255, 0), 2)
    cv2.circle(annotated, (cx, cy), 5, (0, 0, 255), -1)
    cv2.putText(annotated, f"({cx}, {cy})", (cx + 10, cy - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

    cv2.imwrite('E:/glass_git/ML-DL/vision/test_detect.png', annotated)
    print(f"✅ Binary 기반 중심 좌표: ({cx}, {cy}) → 저장됨: binary_detected_front_face.png")
else:
    print("❌ 윤곽선(사다리꼴 포함) 미검출")


❌ 윤곽선(사다리꼴 포함) 미검출
